In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute,QuantumRegister, ClassicalRegister
from numpy.random import randint
from numpy import pi
from time import time
import scipy as sc
import scipy.linalg as la
import json
import math as math
import warnings
warnings.filterwarnings('ignore')


# define noise model for simulation; note the version of qiskit
#from qiskit_aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, coherent_unitary_error,phase_damping_error
from qiskit.providers.aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, coherent_unitary_error,phase_damping_error
from scipy.optimize import curve_fit, minimize
from qiskit.quantum_info import process_fidelity, PTM, diamond_norm

def unitary_error_rx(theta):  #exp(-i theta/2 X)
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], [-1j*np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_rz(theta):  #exp(-i theta/2 Z)
    return np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

def unitary_error_ry(theta):  #exp(-i theta/2 Y)
    return np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])

# generate noise model; 'ps': probability of stochastic noise; 'angle': angle of unitary error; 'axis': the axis of unitary error
def get_noise_model(ps=0.001,angle=0.01,axis='z'):
    noise_model=NoiseModel()
    
    error_ad= amplitude_damping_error(ps)
    error_pd= phase_damping_error(ps)
    error_st= error_pd.compose(error_ad)
    
    if axis=='z':
        error_u_mat= unitary_error_rz(angle)
    elif axis=='x':
        error_u_mat= unitary_error_rx(angle)
    else:
        error_u_mat= unitary_error_ry(angle)
    
    error_u = coherent_unitary_error(error_u_mat)
    
    error_1q= error_u.compose(error_st)
    noise_model.add_all_qubit_quantum_error(error_1q, ['t','h','u1','u2','u3','I','X','Y','Z','ry','rz','rx'])
    
    
    return noise_model,error_1q

# simulate circuits with given noise model
def noise_sim(circ_list,noise_model,shots=8192):
    result = execute(circ_list, backend=Aer.get_backend('qasm_simulator'),noise_model=noise_model, shots=shots,optimization_level=0).result()
    count_list=[result.get_counts(circ) for circ in circ_list]
    exp_list=[count.get('0',0)/shots for count in count_list]
    
    return exp_list

# actual process infidelity and stochastic infidelity of noise 
def act_error(error_1q,target_mat):
    pf= process_fidelity(error_1q)
    noisy_gate=error_1q.compose(coherent_unitary_error(target_mat))
    ptm=PTM(noisy_gate)
    evals, evecs=np.linalg.eig(ptm)
    stof= np.sqrt(np.mean(np.square(np.abs(evals))))
    
    return 1-pf, 1-stof


#define matrix pencil data processing technique; this code is from the paper [spectral quantum tomography] with some modifications.
def matrix_pencil(time_series_data,L ,N_poles,cutoff=10**(-10)):
        """Computes a decomposition into exponentially decaying oscillations of a given time series.
        Input: time_series_data: a list of floats corresponding to the state of the system at fixed time intervals.
                NOTE: It is important this timeseries starts at t=0 (k=0), the method as implemented can't deal with timeshifts and may fail quietly
               L: A matrix pencil parameter. Set between 1/2 and 2/3 of len(time_series_data) 
               N_poles: The number of poles the data can be decomposed into. Choosing this number too small will lead to bad fits so act with care.
               cutoff: a cut-off for the smallest possible relative amplitude a poles can contribute with, standard is 10^(-2). Here we set it to a very low value,because we know the num of poels.
        Ouput: poles: a scipy array of the poles (the oscillating bits).
                amplitudes: a scipy array of amplitudes corresponding to the poles.
                
                """      
        #Compute the length of the data series and store it in N
        N = len(time_series_data)
       
        #Compute the Hankel matrix of the data
        Y = sc.matrix([time_series_data[i:L+i+1] for i in range(0,N-L)]) 
        
        #Take the singular value decomposition of the data Hankel matrix
        U,S,Vh = sc.linalg.svd(Y) 
        Vh =sc.matrix(Vh)
        U = sc.matrix(U)
        if S[2]/S[3]>10: # if the third singular value is ten times larger than the fourth, set the N_poles=3
            N_poles=3       
        #The ESPRIT method includes a filter step that gets rid of small singular values.
        # Since for us the number of poles is known I just retain the N_poles largest singular values and corresponding right eigenspace.
        #If N_poles is so large that it starts to include nonsense values we let the cutoff parameter set the number of relevant poles instead.
        #We choose to retain only the singular values s such that s>s_max * cutoff where s_max is the largest singualr value
        Scutoff = S[S>cutoff*S[0]]
        if len(Scutoff)<N_poles:

            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(len(Scutoff))] for j in range(N-L)])
            Vhprime = Vh[0:len(Scutoff),:]
        else:
            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(N_poles)] for j in range(N-L)])
            Vhprime = Vh[0:N_poles,:]
        
        #Compute the shifted matrices for the matrix pencil.
        Vhprime1 = Vhprime[:,0:-1]
        Vhprime2 = Vhprime[:,1:]
        
        #Compute the solution of the matrix pencil (via SVD)
        Y = la.pinv(Vhprime1.H)*Vhprime2.H
        poles, vecs = sc.linalg.eig(Y)
        for i in range(len(poles)):
            amp= np.abs(poles[i])
            if amp>1:
                poles[i]= poles[i]/amp
        
        #Compute the amplitudes by least squares optimization
        Z = sc.matrix([poles**k for k in range(N)])
        amplitudes = la.lstsq(Z,sc.matrix(time_series_data).transpose())
        ampls = sc.array([a[0] for a in amplitudes[0]])
        amp_max= np.max(np.abs(ampls))
        
        poles_p=[]
        ampls_p=[]
        for i in range(len(poles)):
            if np.abs(ampls[i])/amp_max>0.05:  # discard the poles with very small amplitudes
                poles_p.append(poles[i])
                ampls_p.append(ampls[i])
                
        #return the poles and amplitudes as scipy arrays
        return poles_p, ampls_p,amplitudes,S

# process the poles from matrix_pencil method to obtain 'process infidelity', 'sotchastic infidelity', and rotational angle error
def result_mp(poles,target_phase=0.25*pi,rep=1):
    if len(poles)<4:
        poles= np.append(poles,1)
    amp= np.abs(poles)
    target_phase_p= (target_phase*rep)%(2*pi)
    if target_phase_p-pi> 10**-4:
        target_phase_p= target_phase_p-2*pi
    phase= np.angle(poles)
    phase_dif= np.abs(phase-target_phase_p)
    angle_index= np.argmin(phase_dif)
    angle_error= (phase-target_phase_p)[angle_index]        
    phase_dif2= np.abs(np.abs(phase)-np.abs(target_phase_p))
    index_com= np.argwhere(phase_dif2>np.abs(angle_error))

    res=[]
    angle_error= angle_error/rep
    if np.abs(np.abs(target_phase)-pi)<10**-4:
        angle_error= np.abs(angle_error)   # if rotational angle of target gate is pi, this method only gives the magnitude of angle error
    res.append(angle_error)
    res.append((amp[angle_index])**(1/rep))
    for i in index_com:
        res.append((amp[i[0]])**(1/rep))
    r= 1-(0.5*res[1]*np.cos(res[0])+0.25*(res[2]+res[3]))
    u= 1-np.sqrt((2*res[1]**2+res[2]**2+res[3]**2)/4)
    return [r,u, angle_error], res


class Csb_q1rotation:
      #'rot_angle': rotaion angle of target gate; 'rot_axis':rotational axis
    def __init__(self, rot_angle= pi/4, rot_axis='z'):   
        self.rot_angle = rot_angle
        self.axis = rot_axis
        self.phase = rot_angle
    
    # 'len_list': the list of circuit length; 'ini_mode': the inital state; 'rep':the number of repeats of the target gate
    def benchmark_circ(self,len_list,ini_mode='x',rep=1):
        circ_list=[]
        for lc in len_list:
            qc= QuantumCircuit(1,1)
            if ini_mode=='x':
                qc.h(0)
            elif ini_mode=='y':
                qc.h(0)
                qc.s(0)
            elif ini_mode=='z':
                qc.x(0)
            qc.barrier()
            
            for k in range(lc*rep):
                if self.axis=='x':
                    qc.rx(self.rot_angle,0)
                elif self.axis=='y':
                    qc.ry(self.rot_angle,0)
                if self.axis=='z':
                    qc.rz(self.rot_angle,0)
            qc.barrier()

            if ini_mode=='x':
                qc.h(0)
            elif ini_mode=='y':
                qc.sdg(0)
                qc.h(0)
            elif ini_mode=='z':
                qc.x(0)
            qc.barrier()
            qc.measure(0,0)
            circ_list.append(qc)
        return circ_list
    
    # the full list of circuits for CSB;
    # 'len_max': the maximum length of the list of circuits; 'rep': the number of repeats of the target gate
    def full_circs(self,len_max=50,rep=1):
        self.rep= rep
        if self.axis=='x':
            mode_list=['z','x']
        elif self.axis=='y':
            mode_list=['z','y']
        elif self.axis =='z':
            mode_list= ['x','z']
    
        len_list=[i for i in range(0,len_max+1)]
        circs_list=[]
        for mode in mode_list:
            circs_list = circs_list+ self.benchmark_circ(len_list,ini_mode=mode,rep=rep)
        return circs_list
    
    # input: the list of measured probabilities of circuits;
    # output: [process infidelity, stochastic infidelity, angle error]; diagonal entries of the pure noise channel
    def result(self,data):
        target_phase=self.phase
        data1,data2= np.split(np.array(data),2)
        data_com= data1+ data2
        len_data= len(data_com)
        
        poles, amps,amplitudes,S = matrix_pencil(data_com,L= math.ceil(len_data*3/5),N_poles=4,cutoff=10**-10)
        results,spec = result_mp(poles,target_phase= target_phase, rep= self.rep)
        
        return results, spec
    
    # the function doing numerical simulation
    def simulation(self,noise_model,len_max=50,shots=10**4,rep=1):
        circs=self.full_circs(len_max=len_max,rep=rep)
        data=[]
        data=noise_sim(circs,noise_model=noise_model,shots=shots)
            
        res,spec=self.result(data)
        return res, spec,data

In [ ]:
# simulation of t gate under varying stochastic noise and fixed unitary error
ts=time()
ps_list=[0.001,0.002,0.005,0.007,0.01,0.02,0.05,0.07,0.1]
rep_list=[7 for i in range(5)]+ [1 for i in range(4)]  # if ps<=0.01, rep=7 (other values are also ok but should avoid channel phase=0 or pi and amplify unitary error too much); otherwise, rep=1; 
L_max=50  # 50 is enough, but one can increase 'L_max' further to gain more accuracy
shots=10**4
act_error_list=[]
est_sto=[]
target_mat= unitary_error_rz(pi/4)

for i,ps in enumerate(ps_list):
    noise_model,error_1q= get_noise_model(ps=ps,angle=-0.01,axis='z')
    act_error_list.append(act_error(error_1q,target_mat=target_mat))
    
    for k in range(10):
        csb_t= Csb_q1rotation(rot_angle=pi/4,rot_axis='z')
        res,spec,data=csb_t.simulation(noise_model=noise_model,len_max=L_max,shots=shots,rep=rep_list[i])
        est_sto.append(res)
te=time()
print('time cost=',te-ts)

In [ ]:
ts=time()
angle_list=[0.001,0.002,0.005,0.007,0.01,0.02,0.05,0.07,0.1]
rep_list=[7 for i in range(9)]  # for very weak noise, the more precision needed, which can be achieved by increasing the length of the circuits
L_max=100    # increasing length can be achieved by increasing L_max (also equal to the num of circuits) or increasing 'rep' (the num of target is repeated each time applied; if unitary noise is too strong, 'rep' should be small, otherwise may causing misidentification of eigenvalues)
shots=10**4
act_uni_error_list=[]
est_uni=[]
data_list=[]
target_mat= unitary_error_rz(pi/4)

for i,angle in enumerate(angle_list[0:1]):
    noise_model,error_1q= get_noise_model(ps=0.001,angle=angle,axis='z')
    act_uni_error_list.append(act_error(error_1q,target_mat=target_mat))
    for k in range(1):
        csb_t= Csb_q1rotation(rot_angle=pi/4,rot_axis='z')
        res,spec,data=csb_t.simulation(noise_model=noise_model,len_max=L_max,shots=shots,rep=rep_list[i])
        est_uni.append(res)
        data_list.append(data)
te=time()
print('time cost=',te-ts)